                           Escola Politécnica da Universidade de São Paulo
                                         Data: 13/09/2019
#       PMR3508 - Aprendizado de Máquina e Reconhecimento de Padrões
### Análise e aplicação do k-NN a base *adult*
#### Autor: Lucas Nunes Sequeira

# 1. Preparação dos dados (*Data Prep*)

Nessa primeira etapa, começo importando algumas bibliotecas que usaremos ao longo do desenvolvimento do notebook. Além disso, faremos um tratamento nos **dados faltantes** com o uso da biblioteca *pandas*.

### 1.1 Importação de bibliotecas

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline
plt.style.use('seaborn')

### 1.2 Importação dos dados

Agora, subo o arquivo *train_data.csv* através do *pandas* na forma de **DataFrame**, considerando também valores <font color='red'>*?*</font> como <font color='red'>*Nan*</font>.

In [ ]:
from subprocess import check_output
print(check_output(["ls", "../input/adult-pmr3508"]).decode("utf8"))

In [ ]:
df_train = pd.read_csv("../input/adult-pmr3508/train_data.csv", na_values = '?')
df_train.set_index('Id',inplace=True)
df_train.head()

In [ ]:
print('Forma do DataFrame:', df_train.shape)

### 1.3 Identificando os **dados faltantes** (*missing data*)

Nessa etapa dessa seção, identifico os **dados faltantes** utilizando algumas ferramentas do *pandas*. Para em seguida analisar o tipo, se é aleatório ou não por exemplo.

In [ ]:
total = df_train.isnull().sum().sort_values(ascending = False)
percent = ((df_train.isnull().sum()/df_train.isnull().count())*100).sort_values(ascending = False)
missing_data = pd.concat([total, percent], axis = 1, keys = ['Total', '%'])
missing_data.head()

Podemos observar que os **dados faltantes** concentram-se em 3 colunas: '*occupation*', '*workclass*' e '*native.country*'. Observo as distribuições das colunas, para analisarmos melhor:

In [ ]:
print('occupation:\n')
print(df_train['occupation'].describe())

print('\n\nworkclass:\n')
print(df_train['workclass'].describe())

print('\n\nnative.country:\n')
print(df_train['native.country'].describe())

Para os três casos, devido a alta frequência da moda, utilizarei a moda como valor imputado.

In [ ]:
value = df_train['workclass'].describe().top
df_train['workclass'] = df_train['workclass'].fillna(value)

value = df_train['native.country'].describe().top
df_train['native.country'] = df_train['native.country'].fillna(value)

value = df_train['occupation'].describe().top
df_train['occupation'] = df_train['occupation'].fillna(value)

In [ ]:
total = df_train.isnull().sum().sort_values(ascending = False)
percent = ((df_train.isnull().sum()/df_train.isnull().count())*100).sort_values(ascending = False)
missing_data = pd.concat([total, percent], axis = 1, keys = ['Total', '%'])
missing_data.head()

Como podemos observar, agora nosso banco de dados para o treino está limpo e pronto para ser analisado.

# 2. Análise exploratória dos dados

Nessa etapa, utilizarei as bibliotecas: *matplotlib*, *pandas* e *seaborn* como ferramentas para analisarmos e visualizarmos os dados, e assim tirar algumas conclusões sobre os mesmos.

In [ ]:
cols = ['age', 'fnlwgt', 'education.num', 'capital.gain', 'capital.loss', 'hours.per.week']
sns.set()
sns.pairplot(df_train, vars = cols, hue = 'income')

### 2.1 Descrição dos dados

Observaremos com o uso da função *.describe()* e *.hist()* do *pandas* para clarear nossas primeiras inferências.

In [ ]:
df_train.describe()

De cara observamos que as colunas *capital.gain* e *capital.loss* possuem grupos bem definidos e distantes, para isso, veremos seu histograma:

In [ ]:
plt.figure(figsize=(13, 7))
df_train['capital.gain'].hist(color = 'coral')
plt.xlabel('capital gain')
plt.ylabel('quantity')
plt.title('Capital gain histogram')

In [ ]:
plt.figure(figsize=(13, 7))
df_train['capital.loss'].hist(color = 'coral')
plt.xlabel('capital loss')
plt.ylabel('quantity')
plt.title('Capital loss histogram')

Logo vemos que há uma concentração quase absoluta para valores pequenos, enquanto há poucos, bastante altos. Agora para termos uma noção da curva de idades do banco de dados, avalio com *.hist()*

In [ ]:
plt.figure(figsize=(13, 7))
df_train['age'].hist(color = 'coral')
plt.xlabel('age')
plt.ylabel('quantity')
plt.title('Age histogram')

Além disso, podemos também avaliar com a função *.distplot()* do *seaborn* para compararmos com uma **curva de distribuição**.

In [ ]:
sns.set()
plt.figure(figsize=(13,7))
sns.distplot(df_train['age'], color = 'darkorchid', bins = 70)
plt.ylabel('quantity')
plt.title('Distribution of age')

Vemos que há uma grande quantidade de jovens (20 a 40 anos) nessa população.

In [ ]:
plt.figure(figsize=(13, 7))
df_train['hours.per.week'].hist(color = 'coral')
plt.xlabel('hours per week')
plt.ylabel('quantity')
plt.title('Hours per week histogram')

Concluímos também que em maioria (e em média) os indivídos desse banco de dados trabalham 40 horas por semana (8 horas por dia e finais de semana livre) o que é saudável. No entanto, há uma parcela significativa que excede esse número, analiso a seguir.

In [ ]:
super_work = df_train[df_train['hours.per.week'] > 40]
plt.figure(figsize=(13, 7))
super_work['hours.per.week'].hist(color = 'coral', bins = 5)
plt.xlabel('hours per week')
plt.ylabel('quantity')
plt.title('Hours per week histogram')

E esses '*super trabalhadores*' que trabalham acima de 40 horas por semana constituem um grupo que cai exponencialmente com as horas de trabalho, no entanto, vemos por exemplo que em média eles trabalham:

In [ ]:
mean = super_work['hours.per.week'].describe()['mean']
print('{0} horas por semana ({1} horas por dia com finais de semana livre). O que é algo que já começa a ser bastante desgastante para o trabalhador.'.format(int(mean), int(mean/5)))

### 2.2 Comparação dos dados

Nessa subseção faremos uma análise comparativa entre os dados dos indivídos do banco de dados. Utilizaremos histogramas, gráficos de pizza e boxplots.

Primeiro, crio uma função de comparação para histogramas.

In [ ]:
def compare_histogram(df, obj_var, test_var, obj_labels = None, alpha = 0.7):
    
    if obj_labels is None:
        obj_labels = df[obj_var].unique()
    
    #obj_var = 'income'
    #obj_labels = ['>50K', '<=50K']
    #test_var = 'age' (for example)
    
    temp = []
    n_labels = len(obj_labels)
    for i in range(n_labels):
        temp.append(df[df[obj_var] == obj_labels[i]])
        temp[i] = np.array(temp[i][test_var]).reshape(-1,1)

    fig = plt.figure(figsize= (13,7))
    
    for i in range(n_labels):
        plt.hist(temp[i], alpha = alpha)
    plt.xlabel(test_var)
    plt.ylabel('quantity')
    plt.title('Histogram over \'' + test_var + '\' filtered by \'' + obj_var + '\'')
    plt.legend(obj_labels)

In [ ]:
compare_histogram(df_train, 'income', 'age')

Sobre o histograma acima (*age* e *income*) vemos que há uma distribuição relativamente normal sobre as pessoas que recebem *>50K* com o termo médio próximo de 45 anos; já para pessoas *<=50K* , observa-se que com o aumento da idade, há uma redução na quantidade de pessoas que recebem o valor.

Agora, farei uma análise sobre a distribuição de sexo.

In [ ]:
plt.figure(figsize=(13, 7))
df_train['sex'].value_counts().plot(kind = 'pie')

Vemos que há nos dados, mais homens que mulheres, com relação à esses valores, temos:

In [ ]:
# male = qtd sex == Male
male = df_train[df_train['sex'] == 'Male'].count()[0]

# female = qtd sex == Female
female = df_train.shape[0] - male

In [ ]:
print("Temos {0} homens e {1} mulheres, ou seja, apenas {2:3.2f}% são mulheres.".format(male, female, female*100/(female+male)))

In [ ]:
compare_histogram(df_train, 'income', 'sex')

Sobre o histograma acima, nota-se que há uma grande discrepância percentual (veja os valores abaixo) entre mulheres que recebem *>50K* e homens que recebem o mesmo valor, vemos que há muito mais homens que recebem *>50K*. Esses resultados sugerem que há **desigualdade de gênero** tratando-se do salário.

In [ ]:
# male_income = [qtd > 50K, qtd <= 50K]
male_income = []
temp = df_train[df_train['sex'] == 'Male']
male_income.append(temp[temp['income'] == '>50K'].count()[0])
male_income.append(male-male_income[0])

# female_income = [qtd > 50K, qtd <= 50K]
female_income = []
temp = df_train[df_train['sex'] == 'Female']
female_income.append(temp[temp['income'] == '>50K'].count()[0])
female_income.append(female-female_income[0])

# % of male that has >50K income:
male_over = male_income[0]/male

# % of female that has >50K income:
female_over = female_income[0]/female

In [ ]:
print('Temos que dentre os homens, {0:1.2f}% possuem renda anual superior a 50.000, já dentre as mulheres, temos {1:2.2f}% apenas que possuem renda anual superior a 50.000.'.format(male_over*100, female_over*100))

Ainda sobre a questão de gênero, temos:

In [ ]:
compare_histogram(df_train, 'sex', 'hours.per.week')

O que vemos é que existem muito mais homens (inclusive percentualmente) que trabalham mais de 40 horas por semana, e percentualmente vemos mais mulheres trabalhando menos de 30 horas semanais. Agora analisarei as ocupações.

In [ ]:
female = df_train[df_train['sex'] == 'Female']
male = df_train[df_train['sex'] == 'Male']

In [ ]:
plt.figure(figsize=(13, 7))
male['occupation'].value_counts().plot(kind = 'bar', color = 'purple')
plt.ylabel('quantity')
plt.title('Histogram of male over occupations')

plt.figure(figsize=(13, 7))
female['occupation'].value_counts().plot(kind = 'bar', color = 'coral')
plt.ylabel('quantity')
plt.title('Histogram of female over occupations')

Observamos que há uma nítida diferença entre a empregabilidade dos homens e das mulheres. Além disso observamos também que não há mulheres que trabalham nas *Armed-Forces*, será uma política exclusiva?

Agora, faremos algumas análises étnicas.

In [ ]:
plt.figure(figsize=(13, 7))
df_train['race'].value_counts().plot(kind = 'pie')

O que vemos é uma concentração bastante elevada da quantidade de pessoas brancas nessa população pesquisada.

In [ ]:
compare_histogram(df_train, 'income', 'race')

Observando o histograma acima, podemos de cara visualizar que provavelmente há um percentual maior de pessoas brancas que recebem mais do 50.000 sobre as pessoas negras, amer-indian-eskimo e outros. Os valores seguem abaixo.

In [ ]:
# kind = qtd race == 'unique'
white = df_train[df_train['race'] == 'White'].count()[0]
black = df_train[df_train['race'] == 'Black'].count()[0]
amer = df_train[df_train['race'] == 'Amer-Indian-Eskimo'].count()[0]
other = df_train[df_train['race'] == 'Other'].count()[0]
asian = df_train[df_train['race'] == 'Asian-Pac-Islander'].count()[0]

# kind_income = [qtd > 50K, qtd <= 50K]
white_income = []
temp = df_train[df_train['race'] == 'White']
white_income.append(temp[temp['income'] == '>50K'].count()[0])
white_income.append(white-white_income[0])

black_income = []
temp = df_train[df_train['race'] == 'Black']
black_income.append(temp[temp['income'] == '>50K'].count()[0])
black_income.append(black-black_income[0])

amer_income = []
temp = df_train[df_train['race'] == 'Amer-Indian-Eskimo']
amer_income.append(temp[temp['income'] == '>50K'].count()[0])
amer_income.append(amer-amer_income[0])

asian_income = []
temp = df_train[df_train['race'] == 'Asian-Pac-Islander']
asian_income.append(temp[temp['income'] == '>50K'].count()[0])
asian_income.append(asian-asian_income[0])

other_income = []
temp = df_train[df_train['race'] == 'Other']
other_income.append(temp[temp['income'] == '>50K'].count()[0])
other_income.append(other-other_income[0])

In [ ]:
print('Brancos:\n   {0:1.2f}% recebem mais de 50.000\n'.format(white_income[0]*100/white))
print('Negros:\n   {0:1.2f}% recebem mais de 50.000\n'.format(black_income[0]*100/black))
print('Amer-Indian-Eskimo:\n   {0:1.2f}% recebem mais de 50.000\n'.format(amer_income[0]*100/amer))
print('Asian-Pac-Islander:\n   {0:1.2f}% recebem mais de 50.000\n'.format(asian_income[0]*100/asian))
print('Outros:\n   {0:1.2f}% recebem mais de 50.000'.format(other_income[0]*100/other))

Disso podemos concluir, que há também uma certa **desigualdade racial**, em que etinias não descrita em *Others*, e as etnias *Black* e *Amer-Indian-Eskimo* recebem proporcionalmente menos que as demais. Em seguida, comparemos as duas etinias em maior quantidade (brancos e negros), com relação às ocupações.

In [ ]:
white = df_train[df_train['race'] == 'White']
black = df_train[df_train['race'] == 'Black']

In [ ]:
plt.figure(figsize=(13, 7))
white['occupation'].value_counts().plot(kind = 'bar', color = 'purple')
plt.ylabel('quantity')
plt.title('Histogram of white people over occupations')

plt.figure(figsize=(13, 7))
black['occupation'].value_counts().plot(kind = 'bar', color = 'coral')
plt.ylabel('quantity')
plt.title('Histogram of black people over occupations')

Novamente vemos que há uma grande diferença entre as ocupações para cada etnia. Além disso podemos fazer uma comparação sobre a educação.

In [ ]:
var1 = 'race'
var2 = 'education.num'

data = pd.concat([df_train[var2], df_train[var1]], axis=1)

f, ax = plt.subplots(figsize=(15, 7))

sns.boxplot(x=var1, y=var2, data=data, notch = True)
plt.title('Boxplot of education num over race')

Vemos novamente, brancos e Asian-Pac-Islander, com uma educação em média superior às demais etnias.

Agora, analisaremos os salários das ocupações, para tirarmos mais conclusões.

In [ ]:
over_50k = df_train[df_train['income'] == '>50K']
under_50k = df_train[df_train['income'] == '<=50K']

In [ ]:
plt.figure(figsize=(13, 7))
over_50k['occupation'].value_counts().plot(kind = 'bar', color = 'purple')
plt.ylabel('quantity')
plt.title('Histogram of income over 50K over occupations')

plt.figure(figsize=(13, 7))
under_50k['occupation'].value_counts().plot(kind = 'bar', color = 'coral')
plt.ylabel('quantity')
plt.title('Histogram of income under 50K over occupations')

Observamos que aqueles que recebem acima de 50.000 em maioria são *Exec-managerial*, seguido por *Prof-specialty*, que são ocupações ocupadas em maioria por homens brancos. Já aqueles que recebem menos de 50.000 ocupam em maioria *Adm-clerical*, *Craft-repair*, *Other-service* e *Sales*, que são áreas que empregam maioria mulheres e negros.

Uma análise também interessante é sobre as idades.

In [ ]:
var2 = 'age'
var1 = 'hours.per.week'

data = pd.concat([df_train[var2], df_train[var1]], axis=1)

f, ax = plt.subplots(figsize=(14, 15))

sns.boxplot(x=var1, y=var2, data=data, orient = 'h')
plt.title('Boxplot of age over hours per week')

Aqui vemos que pessoas muito jovens trabalham em média menos de 40 horas por semana, provavelmente pois devem trabalhar meio período. Assim como pessoas mais idosas, em média trabalham também menos de 40 horas por dia.

Uma outra avaliação é sobre a quantidade de tempo de trabalho por semana com relação ao nível de educação das pessoas.

In [ ]:
var2 = 'education'
var1 = 'hours.per.week'

data = pd.concat([df_train[var2], df_train[var1]], axis=1)

f, ax = plt.subplots(figsize=(13, 7))
#ax.set_ylim(0,10000)

order = ['Preschool', '1st-4th', '5th-6th', '7th-8th', '9th', '10th',
         '11th', '12th', 'HS-grad', 'Prof-school', 'Assoc-acdm', 'Assoc-voc',
         'Some-college', 'Bachelors', 'Masters', 'Doctorate']
sns.boxplot(x=var1, y=var2, data=data, order = order)

Vemos que as pessoas com os maiores níveis de educação, normalmente trabalham mais de 40 horas por dia, como aqueles que possuem título de doutorado, mestrado e bacharel; como também os que concluiram uma escola profissionalizante.

# 3. Análise e preprocessamento dos atributos

Nessa seção usaremos bibliotecas como *skitlearn* para fazermos alguns preprocessamento dos dados, como também algumas análises de correlação entre os atributos para otimizarmos nosso aprendizado de máquina no futuro. 

### 3.1 Tratamento dos atributos

Agora, farei um tratamento sobre alguns dos atributos qualitativos e quantitativos de forma a numerar aqueles ordenáveis por exemplo.

#### 3.1.1 Alguns tratamentos de dados

referência: https://www.rdocumentation.org/packages/arules/versions/1.6-3/topics/Adult

*education*

- qualitativo ordenado
    - Preschool < 1st-4th < 5th-6th < 7th-8th < 9th < 10th < 11th < 12th < HS-grad < Prof-school < Assoc-acdm < Assoc-voc < Some-college < Bachelors < Masters < Doctorate
    
*workclass*

- booleano
    - isPrivate
    
*age*
 *   
- quantitativo ordenado
     - Young (0-25) < Middle-aged (26-45) < Senior (46-65) < Old (66+)
     
*hours.per.week*

- quantitativo ordenado
    - Part-time (0-25) < Full-time (25-40) < Over-time (40-60) < Too-much (60+).

*capital.gain* e *capital.loss*

- quantitativo ordenado
    - None (0) < Low (0 - mediana dos valores maiores que zero) and High (> mediana dos valores maiores que zero).

In [ ]:
df_train['income'].value_counts()

In [ ]:
base = df_train

Agora vou dividir os atributos quantitativos e qualitativos:

In [ ]:
base.columns

In [ ]:
quantitative_columns = ['age', 'fnlwgt', 'education.num', 'capital.gain', 'capital.loss', 'hours.per.week']
qualitative_columns = ['education', 'marital.status', 'occupation', 'relationship', 'race',
                       'sex', 'native.country', 'income']

#### 3.1.2 Funções

Algumas declarações de funções para o tratamento dos dados:

In [ ]:
def isprivate(value):
    if value == 'Private':
        return 1
    return 0

def catg(value, categories, ordenation = None):
    if ordenation is not None:
        ordenation = np.arange(0, len(categories))
    for pos in ordenation:
        if value == categories[pos]:
            return pos
    return -1

def equals(value, x):
    for v in x:
        if v == value:
            return 1
    return 0

#### 3.1.3 *workclass*

In [ ]:
base['workclass'].unique()

In [ ]:
# privado: 1 se trabalha para o privado, 0 caso contrario
private = pd.DataFrame({'private': base['workclass'].apply(isprivate)})

#### 3.1.4 *native.country*

In [ ]:
df_train['native.country'].value_counts()

In [ ]:
# usa: 1 se é sul_global, 0 caso contrário
countries = ['Mexico', 'Philippines', 'Puerto-Rico', 'El-Salvador', 'India', 'Cuba', 'Jamaica',
             'South', 'China', 'Dominican-Republic', 'Vietnam', 'Guatemala', 'Columbia', 'Taiwan',
             'Haiti', 'Iran', 'Nicaragua', 'Peru', 'Ecuador', 'Trinadad&Tobago', 'Cambodia',
             'Laos', 'Thailand', 'Yugoslavia', 'Outlying-US(Guam-USVI-etc)', 'Honduras']
sul_global = pd.DataFrame({'sul.global': base['native.country'].apply(equals, args = [countries])})

#### 3.1.5 *education*

Preschool < 1st-4th < 5th-6th < 7th-8th < 9th < 10th < 11th < 12th < HS-grad < Prof-school < Assoc-acdm < Assoc-voc < Some-college < Bachelors < Masters < Doctorate

In [ ]:
base['education'].unique()

In [ ]:
edu_order = [15, 11, 5, 12, 10, 1, 14, 7, 2, 8, 4, 13, 0, 3, 6, 9]
args = [base['education'].unique(), edu_order]
education_classes = pd.DataFrame({'education.classes': base['education'].apply(catg, args = args)})

#### 3.1.6 *hours.per.week*

In [ ]:
aux = pd.cut(base['hours.per.week'], bins = [-1, 25, 40, 60, 200], labels = [0, 1, 2, 3])
hours_per_week_clusters = pd.DataFrame({'hours.per.week.clusters': aux})
hours_per_week_clusters = hours_per_week_clusters.astype(np.int)

#### 3.1.7 *capital.gain* e *capital.loss*

In [ ]:
median = np.median(base[base['capital.gain'] > 0]['capital.gain'])
aux = pd.cut(base['capital.gain'],
             bins = [-1, 0, median, base['capital.gain'].max()+1],
             labels = [0, 1, 2])
capital_gain_clusters = pd.DataFrame({'capital.gain.clusters': aux})
capital_gain_clusters = capital_gain_clusters.astype(np.int)

median = np.median(base[base['capital.loss'] > 0]['capital.loss'])
aux = pd.cut(base['capital.loss'],
             bins = [-1, 0, median, base['capital.loss'].max()+1],
             labels = [0, 1, 2])
capital_loss_clusters = pd.DataFrame({'capital.loss.clusters': aux})
capital_loss_clusters = capital_loss_clusters.astype(np.int)

In [ ]:
new_data = pd.concat([sul_global, private, education_classes, 
                      hours_per_week_clusters, capital_gain_clusters, 
                      capital_loss_clusters], axis = 1)

Por fim, construimos essa pequena tabela de dados que podem vir a ser uteis.

In [ ]:
new_data.head()

In [ ]:
aux = base['income'].apply(equals, args = [['>50K']])

aux = pd.concat([new_data, pd.DataFrame({'income': aux})], axis = 1)

new = aux.astype(np.int)
aux.head()

A seguir, uma simples visualização com uma matriz de correlação para termos uma ideia das contribuições sobre *income*.

In [ ]:
corr_mat = aux.corr()
corr_mat
sns.set()
plt.figure(figsize=(10,8))
sns.heatmap(corr_mat, annot=True)

### 3.2 Junção da base com os valores construídos e seleção de alguns atributos por intuição

In [ ]:
base = base.drop(['fnlwgt', 'education', 'sex', 'native.country', 'workclass', 'marital.status'], axis = 1)
base.columns

In [ ]:
base = pd.concat([new_data, base], axis = 1)

In [ ]:
base.head()

### 3.3 Encoding dos dados classicatórios

Utilizo o LabelEncoder() para fazer o tratamento dos dados:

In [ ]:
from sklearn import preprocessing as prep

names = ['occupation', 'relationship', 'race']
enc_x = []
for i in range(len(names)):
    enc_x.append(prep.LabelEncoder())
enc_y = prep.LabelEncoder()

In [ ]:
i = 0
for name in names:
    base[name] = enc_x[i].fit_transform(base[name])
    i += 1

base['income'] = enc_y.fit_transform(base['income'])

In [ ]:
base.head()

### 3.4 Matriz de correlação

Agora, farei uma análise sobre a correlação entre os atributos, para isso utilizarei da matriz de correlação do próprio **DataFrame** associado ao *seaborn*.

In [ ]:
aux = base.astype(np.int)

corr_mat = aux.corr()
f, ax = plt.subplots(figsize=(20, 13))
sns.heatmap(corr_mat, vmax=.7, square=True, cmap="coolwarm", annot = True)

### 3.3 Remoção de atributos pouco relevantes

Utilizo a matriz de correlação acima, para visualisar e reduzir a dimenção do problema excluindo atributos pouco relevantes.

In [ ]:
unselected_columns = []
unselected_columns.append('capital.loss')
unselected_columns.append('capital.gain')
unselected_columns.append('sul.global')
unselected_columns.append('private')
unselected_columns.append('education.classes')
unselected_columns.append('hours.per.week.clusters')

base = base.drop(unselected_columns, axis = 1)
base.head()

In [ ]:
aux = base.astype(np.int)

In [ ]:
corr_mat = aux.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corr_mat, vmax=.7, square=True, cmap="coolwarm")

In [ ]:
base.head()

# 4. Agrupamento por K-NN

Nessa última etapa, utilizarei a biblioteca *sklearn* para utilização do método de aprendizado **k-NN**. Nesse processo, farei o uso da **validação cruzada** para avaliar o desempenho do modelo.

### 4.1 Escalonamento dos dados

Primeiro farei um tratamento sobre os dados, de forma a escaloná-los utilizando *StandardScaler*.

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
base.shape

In [ ]:
X = base.drop(['income'], axis = 1)
y = base['income']

In [ ]:
scaler_x = StandardScaler()

X = scaler_x.fit_transform(X)

### 4.3 Definição do hiperparâmetro k do k-NN

Primeiro, avaliaremos o comportamento do aprenzidado para k = 1, ..., 29 para que possamos identificar o melhor valor de k para utilizarmos para prever os valores da base de teste a ser importada. Essa avaliação, será feita com a utilização da validação cruzada com 5 *folds*, e assim será possível determinar o melhor hiperparâmetro k que se aplicará ao nosso problema.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
scores_mean = []
scores_std = []

k_lim_inf = 1
k_lim_sup = 30

folds = 5

k_max = None
max_std = 0
max_acc = 0

i = 0
print('Finding best k...')
for k in range(k_lim_inf, k_lim_sup):
    
    KNNclf = KNeighborsClassifier(n_neighbors=k, p = 1)
    
    score = cross_val_score(KNNclf, X, y, cv = folds)
    
    scores_mean.append(score.mean())
    scores_std.append(score.std())
    
    if scores_mean[i] > max_acc:
        k_max = k
        max_acc = scores_mean[i]
        max_std = scores_std[i]
    i += 1
    if not (k%3):
        print('   K = {0} | Best CV acc = {1:2.2f}% +/-{3:4.2f}% (best k = {2})'.format(k, max_acc*100, k_max, max_std*100))
print('\nBest k: {}'.format(k_max))

In [ ]:
plt.figure(figsize=(15, 7))
plt.errorbar(np.arange(k_lim_inf, k_lim_sup), scores_mean, scores_std,
             marker = 'o', markerfacecolor = 'purple' , linewidth = 3,
             markersize = 10, color = 'coral', ecolor = 'purple', elinewidth = 1.5)


yg = []
x = np.arange(0, k_lim_sup+1)
for i in range(len(x)):
    yg.append(max_acc)
plt.plot(x, yg, '--', color = 'purple', linewidth = 1)
plt.xlabel('k')
plt.ylabel('accuracy')
plt.title('KNN performed on several values of k')
plt.axis([0, k_lim_sup, min(scores_mean) - max(scores_std), max(scores_mean) + 1.5*max(scores_std)])

Acima vemos o gráfico dos erros médios sobre a validação cruzada para diferentes valores de k, em que foi possível determinar o melhor k que satisfaz nosso problema.

### 4.4 Treinamento do k-NN

Agora, faremos o treinamento do k-NN utilizando nossa base de treino, em seguida inferimos sobre nossa acurácia da base de testes.

In [ ]:
k = k_max

KNNclf = KNeighborsClassifier(n_neighbors=k, p = 1)
KNNclf.fit(X, y)

# 5. Predição dos valores sem classe

Nessa última etapa, utilizaremos o nosso classificador treinado para predizer sobre as classes do banco de dados teste que não possui valor de classe atribuído aos indivíduos.

In [ ]:
df_test = pd.read_csv("../input/adult-pmr3508/test_data.csv", na_values='?')
df_test.set_index('Id', inplace = True)
df_test.head()

### 5.1 Adição das colunas extra

In [ ]:
# capital.gain.cluster
median = np.median(df_test[df_test['capital.gain'] > 0]['capital.gain'])
aux = pd.cut(df_test['capital.gain'],
             bins = [-1, 0, median, df_test['capital.gain'].max()+1],
             labels = [0, 1, 2])
capital_gain_clusters = pd.DataFrame({'capital.gain.clusters': aux})
capital_gain_clusters = capital_gain_clusters.astype(np.int)

# capital.loss.cluster
median = np.median(df_test[df_test['capital.loss'] > 0]['capital.loss'])
aux = pd.cut(df_test['capital.loss'],
             bins = [-1, 0, median, df_test['capital.loss'].max()+1],
             labels = [0, 1, 2])
capital_loss_clusters = pd.DataFrame({'capital.loss.clusters': aux})
capital_loss_clusters = capital_loss_clusters.astype(np.int)

new_data = pd.concat([capital_gain_clusters, capital_loss_clusters], axis = 1)

In [ ]:
features = ['age', 'education.num', 'occupation', 'relationship', 'race', 'hours.per.week']

base_test = pd.concat([new_data, df_test[features]], axis = 1)

In [ ]:
base_test.head()

### 5.2 Verificando dados faltantes

In [ ]:
total = base_test.isnull().sum().sort_values(ascending = False)
percent = ((base_test.isnull().sum()/base_test.isnull().count())*100).sort_values(ascending = False)
missing_data = pd.concat([total, percent], axis = 1, keys = ['Total', '%'])
missing_data.head()

In [ ]:
value = base_test['occupation'].describe().top
base_test['occupation'] = base_test['occupation'].fillna(value)

In [ ]:
total = base_test.isnull().sum().sort_values(ascending = False)
percent = ((base_test.isnull().sum()/base_test.isnull().count())*100).sort_values(ascending = False)
missing_data = pd.concat([total, percent], axis = 1, keys = ['Total', '%'])
missing_data.head()

### 5.3 Encoding

In [ ]:
names = ['occupation', 'relationship', 'race']

i = 0
for name in names:
    base_test[name] = enc_x[i].transform(base_test[name])
    i += 1

In [ ]:
base_test.head()

### 5.4 Predição da base teste

Enfim, predizemos as classes:

In [ ]:
X_prev = scaler_x.transform(base_test.values)

In [ ]:
temp = KNNclf.predict(X_prev)

temp = enc_y.inverse_transform(temp)
temp = {'Income': temp}
predictions = pd.DataFrame(temp)

In [ ]:
predictions.head()

### 5.5 Submissão do resultado

Por fim, escrevo os resultados em **submission.csv**

In [ ]:
predictions.to_csv("submission.csv", index = True, index_label = 'Id')

# Exercício Extra - Costa Rican Household Poverty Levels

# 6. Dados

### 6.1 Importando bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

### 6.2 Importando dados

In [ ]:
from subprocess import check_output
print(check_output(["ls", "../input/"]).decode("utf8"))

In [ ]:
df_train = pd.read_csv('../input/rs-costa-rican-household-poverty-level-prediction/train.csv')
df_train.set_index('ID_num', inplace = True)
df_train.head()

### 6.3 Seleção de dados por intuição

In [ ]:
features = ['idhogar', 'parentesco1', 'escolari', 'SQBmeaned', 'hogar_nin', 'hogar_total', 'area1',
            'lugar1', 'cielorazo', 'pisonotiene', 'v14a', 'abastaguano', 'v2a1',
            'hacdor', 'meaneduc', 'SQBovercrowding', 'abastaguadentro',
            'tipovivi1', 'Target']
base = df_train[features]

### 6.4 Filtragem por parentesco e idhogar

In [ ]:
base = base[base['parentesco1'] == 1]
base.shape

base = base.drop(['idhogar', 'parentesco1'], axis = 1)

In [ ]:
base = base.astype(np.float)
print('shape:', base.shape)
base.head()

In [ ]:
corrmat = base.corr()
sns.set()
plt.figure(figsize=(13,9))
sns.heatmap(corrmat)

# 7. Limpeza dos dados

### 7.1 Determinando dados faltantes

In [ ]:
total = base.isnull().sum().sort_values(ascending = False)
percent = ((base.isnull().sum()/base.isnull().count())*100).sort_values(ascending = False)
missing_data = pd.concat([total, percent], axis = 1, keys = ['Total', '%'])
missing_data.head()

### 7.2 Fazendo alterações sobre a base

In [ ]:
base = base.drop(['v2a1'], axis = 1)

#### 7.2.1 *SQBmeaned*

In [ ]:
base['SQBmeaned'].plot(kind = 'box')

In [ ]:
col = 'SQBmeaned'
base[col] = base[col].fillna(base[col].describe().mean())

#### 7.2.1 *meaneduc*

In [ ]:
base['meaneduc'].plot(kind = 'box')

In [ ]:
col = 'meaneduc'
base[col] = base[col].fillna(base[col].describe().mean())

In [ ]:
total = base.isnull().sum().sort_values(ascending = False)
percent = ((base.isnull().sum()/base.isnull().count())*100).sort_values(ascending = False)
missing_data = pd.concat([total, percent], axis = 1, keys = ['Total', '%'])
missing_data.head()

# 8. Visualização dos dados


In [ ]:
def labels(x):
    if x == 1.0:
        return 'extreme poverty'
    if x == 2.0:
        return 'moderate poverty'
    if x == 3.0:
        return 'vulnerable households'
    return 'non vulnerable households'

In [ ]:
base['Target'] = base['Target'].apply(labels)

### 8.1 Pairplot

In [ ]:
cols = ['escolari', 'SQBmeaned', 'area1', 'lugar1', 'hogar_nin', 'hogar_total', 'SQBovercrowding']

sns.set()
sns.pairplot(base, hue = 'Target', vars = cols)

### 8.2 Boxplot

In [ ]:
var2 = 'escolari'
var1 = 'Target'

f, ax = plt.subplots(figsize=(14, 8))

sns.boxplot(x=var1, y=var2, data=base, order = ['extreme poverty', 'moderate poverty', 'vulnerable households', 'non vulnerable households'])
plt.title('Boxplot of escolari over Target')

Maior a escolaridade, maior o nível social (em média)

In [ ]:
var2 = 'hogar_nin'
var1 = 'Target'

f, ax = plt.subplots(figsize=(14, 8))

sns.boxplot(x=var1, y=var2, data=base, order = ['extreme poverty', 'moderate poverty', 'vulnerable households', 'non vulnerable households'])
plt.title('Boxplot of hogar_nin over Target')

Nivel socioeconômico descresce com aumento de crianças

In [ ]:
var2 = 'SQBmeaned'
var1 = 'Target'

f, ax = plt.subplots(figsize=(14, 8))

sns.boxplot(x=var1, y=var2, data=base, order = ['extreme poverty', 'moderate poverty', 'vulnerable households', 'non vulnerable households'])
plt.title('Boxplot of escolari over Target')

Com aumento do nível socio-econômico, temos também o aumento médio do tamanho das habitações

### 8.3 Histogramas

In [ ]:
base.hist(column='cielorazo', by ='Target', figsize=(10,10), color = 'coral')

Percentual de casas com teto aumenta com o nível socio-econômico

In [ ]:
base.hist(column='pisonotiene', by ='Target', figsize=(10,10), color = 'coral')

Aparentemente o percentual de casas sem chão decresce com aumento do nível socio-econômico, mas são poucos casos.

In [ ]:
base.hist(column='v14a', by ='Target', figsize=(10,10), color = 'coral')

O percentual de casas sem banheiro parece aumentar também com o aumento do nível socioeconômico, mas são poucos exemplos.

# 9. Imputar dados

~~~
target = base['Target']
aux1 = pd.DataFrame({'Target | %': target.value_counts(normalize=True)})
aux1
~~~

**Vemos nesse caso que a distribuição dos dados desfavorece a presença de *extreme poverty*

### 9.1 Atributo: *pisonotiene*

~~~
sem_chao = base[base['pisonotiene'] == 1.0]
sem_chao.shape
~~~

~~~
batch = 32
n_batchs = int(500/batch)
base_aux = sem_chao.sample(batch)
for i in range(n_batchs):
    base_aux = pd.concat([base_aux, sem_chao.sample(batch)], axis = 0)
~~~

~~~
base = pd.concat([base, base_aux], axis = 0)
~~~

~~~
base.hist(column='pisonotiene', by ='Target', figsize=(10,10), color = 'coral')
~~~

# 10. Seleção e escalonamento dos dados

### 10.1 Seleção dos melhores atributos

In [ ]:
corrmat = base.corr()
sns.set()
plt.figure(figsize=(13,10))
sns.heatmap(corrmat)

In [ ]:
base = base.drop(['cielorazo', 'v14a', 'abastaguano', 'tipovivi1', 'hacdor'], axis = 1)

In [ ]:
base.head()

### 10.2 Escalonamento dos dados

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
X = base.drop('Target', axis = 1)
y = base['Target']

In [ ]:
scaler_x = StandardScaler()

X = scaler_x.fit_transform(X)

# 11. Classificador k-NN

### 11.1 Seleção dos hiperparâmetros por CV

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
scores_mean = []
scores_std = []

k_lim_inf = 1
k_lim_sup = 36

folds = 5

k_max = None
max_acc = 0

i = 0
print('Finding best k...')
for k in range(k_lim_inf, k_lim_sup):
    
    KNNclf = KNeighborsClassifier(n_neighbors=k, weights='distance', metric='manhattan', p=2)
    
    score = cross_val_score(KNNclf, X, y, cv = folds)
    
    scores_mean.append(score.mean())
    scores_std.append(score.std())
    
    if scores_mean[i] > max_acc:
        k_max = k
        max_acc = scores_mean[i]
    i += 1
    if not (k%3):
        print('   K = {0} | Best CV acc = {1:2.2f}% (best k = {2})'.format(k, max_acc*100, k_max))
print('\nBest k: {}'.format(k_max))

In [ ]:
plt.figure(figsize=(15, 7))
plt.errorbar(np.arange(k_lim_inf, k_lim_sup), scores_mean, scores_std,
             marker = 'o', markerfacecolor = 'purple' , linewidth = 3,
             markersize = 10, color = 'coral', ecolor = 'purple', elinewidth = 1.5)


yg = []
x = np.arange(0, k_lim_sup+1)
for i in range(len(x)):
    yg.append(max_acc)
plt.plot(x, yg, '--', color = 'purple', linewidth = 1)
plt.xlabel('k')
plt.ylabel('accuracy')
plt.title('KNN performed on several values of k')
plt.axis([0, k_lim_sup, min(scores_mean) - max(scores_std), max(scores_mean) + 1.5*max(scores_std)])